In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('order_products__train.csv',
                 header = 0,
                 sep = ',')

In [6]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
